## Import libraries

In [1]:
%autosave 0
import numpy as np
from PIL import Image
from mss import mss
import cv2
import time
import pyautogui
from skimage.measure import compare_ssim as ssim

from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
import keras.backend as K

Autosave disabled


Using TensorFlow backend.


## Set variables

In [2]:
sct = mss()
monitor={'top':50,'left':60,'width':650,'height':485}

width = 224
height = 224

i = 0
old_screen = np.zeros(shape=(224,224,4), dtype='uint8')

## Reset GPU memory if resource exhausted error

In [3]:
def limit_mem():
    K.get_session().close()
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))

limit_mem()

## Driving commands

In [4]:
def forward():
    pyautogui.keyDown('w')
    pyautogui.keyUp('w')

In [5]:
def left():
    pyautogui.keyDown('a')
    pyautogui.keyDown('w')
    pyautogui.keyUp('a')
    pyautogui.keyUp('w')

In [6]:
def right():
    pyautogui.keyDown('d')
    pyautogui.keyDown('w')
    pyautogui.keyUp('d')
    pyautogui.keyUp('w')

In [7]:
def stuck():
    pyautogui.keyDown('s')
    pyautogui.keyDown('a')
    time.sleep(2)
    pyautogui.keyUp('s')
    pyautogui.keyUp('a')

## Screen grab and process

In [8]:
def grab_screen():
    screen = np.array(sct.grab(monitor))
    screen = cv2.resize(screen, (width, height))
    return screen

## Main function

In [10]:
# load trained model
model = load_model('keras_models/gtavc_rev.h5')

print("running")

# pause to change to GTA windows
time.sleep(2)

while(True):

    screen = grab_screen()
    
    # check for stuck vehicle every 5 frames
    i += 1
    if (i % 5 == 0):
        img_var = ssim(old_screen, screen, multichannel=True)
        
        print(img_var)
        if (img_var > 0.70):
            stuck()
        old_screen = screen
    
    screen_img = cv2.cvtColor(screen, cv2.COLOR_BGR2RGB)
    screen_img = Image.fromarray(screen_img, 'RGB')
    
    # preprocess image for neural network
    x = image.img_to_array(screen_img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    
    prediction = model.predict(x)
    #print(prediction)

    max_prediction = prediction.argmax()

    if (max_prediction == 0 or max_prediction == 2):
        forward()
        #print("forward")
        
    elif (max_prediction == 1):
        left()
        #print("left")
    
    elif (max_prediction == 3):
        right()
        #print("right")

    cv2.imshow('window', screen)

    if (cv2.waitKey(25) & 0xFF == ord('q')):
        cv2.destroyAllWindows()
        break

running
0.451181808519
0.88337831673
0.453160131026
0.493658238039
0.533129196177
0.555926927456
0.630568400948
0.636544617602
0.609628960143
0.546261329274
0.456662256111
0.483570241037
0.623262103718
0.593797476696
0.57240823463
0.572097403777
0.605886392563
0.577154631156
0.550587735284
0.516460661944
0.500749240291
0.570699236574
0.512548534114
0.568254588425
0.630977074684
0.408795517316
0.423594107099
0.508740405075
0.624694093851
0.543380607825
0.561500516728
0.582088786989
0.537681640808
0.556149422077
0.599002898406
0.666371684742
0.603947082165
0.57704585992
0.576230360859
0.614438196879
0.603986306937
0.510051403361
0.506837873923
0.57167765225
0.525492111318
0.544345295147
0.601516476348
0.531891807137
0.512326031186
0.599626568458
0.606109222693
0.631674264047
0.633583338832
0.562262838003
0.54864810123
0.561646369376
0.569331677714
0.592979401028
0.625670316948
0.640319836146
0.620357812223
0.55801489445
0.562026796346
0.543619794678
0.497081873692
0.546766002474
0.582691